In [452]:
import pandas as pd
#####데이터 가공하는 첫번째 과정 ######
input_file = '/content/drive/MyDrive/Colab Notebooks/P프로젝트/bmi.csv'
output_file = '/content/drive/MyDrive/Colab Notebooks/P프로젝트/gender.csv'
df = pd.read_csv(input_file)

# Gender 열 추가 (Height 기준으로 1.70m 밑은 Female, 이상은 Male)
df['Gender'] = df['Height'].apply(lambda x: 'Female' if x < 1.70 else 'Male')

print(df.head(5))

df.to_csv(output_file, index=False)
print(f"가공 완료. 데이터가 '{output_file}'에 저장되었습니다.")


   Age  Height  Weight        Bmi       BmiClass  Gender
0   61    1.85  109.30  31.935720  Obese Class 1    Male
1   60    1.71   79.02  27.023700     Overweight    Male
2   60    1.55   74.70  31.092612  Obese Class 1  Female
3   60    1.46   35.90  16.841809    Underweight  Female
4   60    1.58   97.10  38.896010  Obese Class 2  Female
가공 완료. 데이터가 '/content/drive/MyDrive/Colab Notebooks/P프로젝트/gender.csv'에 저장되었습니다.


#1. 라이브러리 및 데이터 경로 설정

In [453]:
# 필요한 라이브러리 불러오기
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# 데이터 파일 경로
bmi_data_path = '/content/drive/MyDrive/Colab Notebooks/P프로젝트/gender.csv'
food_data_path = '/content/drive/MyDrive/Colab Notebooks/P프로젝트/final_food_data.csv'

# 데이터 읽기
bmi_data = pd.read_csv(bmi_data_path)
food_data = pd.read_csv(food_data_path)

# 데이터 확인
print("BMI 데이터 예시:")
print(bmi_data.head())
print("\n식품 데이터 예시:")
print(food_data.head())


BMI 데이터 예시:
   Age  Height  Weight        Bmi       BmiClass  Gender
0   61    1.85  109.30  31.935720  Obese Class 1    Male
1   60    1.71   79.02  27.023700     Overweight    Male
2   60    1.55   74.70  31.092612  Obese Class 1  Female
3   60    1.46   35.90  16.841809    Underweight  Female
4   60    1.58   97.10  38.896010  Obese Class 2  Female

식품 데이터 예시:
       식품명 식품대분류명  에너지(kcal)  탄수화물(g)  단백질(g)  지방(g)  당류(g)  식품중량
0  국밥_돼지머리     밥류        137    15.94    6.70   5.16   0.16  900g
1  국밥_순대국밥     밥류         75    10.38    3.17   2.28   0.17  900g
2   국밥_콩나물     밥류         52    10.93    1.45   0.24   0.00  780g
3      기장밥     밥류        166    36.77    3.44   0.57   0.00  200g
4       김밥     밥류        140    19.98    4.84   4.55   0.00  230g


# 2.BMI 데이터 전처리


In [454]:
# BMR 계산 함수 (Harris-Benedict 공식)
def calculate_bmr(weight, height, age, gender):
    if gender == 'Male':
        return 88.362 + (13.397 * weight) + (4.799 * height) - (5.677 * age)
    else:
        return 447.593 + (9.247 * weight) + (3.098 * height) - (4.330 * age)

# BMI 상태와 목표에 따른 칼로리 목표 설정 함수
def get_calorie_target(bmi_class, bmr, goal):
    if goal == '다이어트':
        return bmr - 500 if bmi_class != '저체중' else bmr - 300
    elif goal == '유지':
        return bmr
    elif goal == '벌크업':
        return bmr + 500 if bmi_class != '비만' else bmr + 300

# BMR 및 칼로리 목표 계산
bmi_data['BMR'] = bmi_data.apply(lambda row: calculate_bmr(row['Weight'], row['Height'] * 100, row['Age'], row['Gender']), axis=1)
bmi_data['Calorie_Target'] = bmi_data.apply(lambda row: get_calorie_target(row['BmiClass'], row['BMR'], '다이어트'), axis=1)

print("BMR 및 칼로리 목표 추가된 BMI 데이터:")
print(bmi_data.head())


BMR 및 칼로리 목표 추가된 BMI 데이터:
   Age  Height  Weight        Bmi       BmiClass  Gender         BMR  \
0   61    1.85  109.30  31.935720  Obese Class 1    Male  2094.17210   
1   60    1.71   79.02  27.023700     Overweight    Male  1627.00194   
2   60    1.55   74.70  31.092612  Obese Class 1  Female  1358.73390   
3   60    1.46   35.90  16.841809    Underweight  Female   972.06830   
4   60    1.58   97.10  38.896010  Obese Class 2  Female  1575.16070   

   Calorie_Target  
0      1594.17210  
1      1127.00194  
2       858.73390  
3       472.06830  
4      1075.16070  


#2-1. 식품 데이터 전처리

In [455]:
# 식품 데이터 전처리
food_data['식품중량'] = food_data['식품중량'].str.replace('ml', 'g').str.replace('m', '').str.replace('g', '')
food_data['식품중량'] = pd.to_numeric(food_data['식품중량'], errors='coerce')
food_data = food_data.dropna(subset=['식품중량'])

# 100g 기준으로 영양소 계산
food_data['칼로리_100g'] = (food_data['에너지(kcal)'] / food_data['식품중량']) * 100
food_data['탄수화물_100g'] = (food_data['탄수화물(g)'] / food_data['식품중량']) * 100
food_data['단백질_100g'] = (food_data['단백질(g)'] / food_data['식품중량']) * 100
food_data['지방_100g'] = (food_data['지방(g)'] / food_data['식품중량']) * 100

# 이상치 제거 (칼로리 800 이상 제외)
filtered_food_data = food_data[food_data['칼로리_100g'] < 800]

# 음식 분류 추가: 밥류, 반찬류, 디저트류
def classify_food(row):
    if any(x in row['식품대분류명'] for x in ["밥류", "면 및 만두류"]):  # 밥류에 면 및 만두류 추가
        return "밥류"
    elif any(x in row['식품대분류명'] for x in ["국 및 탕류", "찌개 및 전골류"]):  # 국 기준
        return "국류"
    elif any(x in row['식품대분류명'] for x in ["전·적 및 부침류","조림류", "나물·숙채류", "튀김류", "구이류", "장류", "양념류", "찜류", "볶음류","튀김류","생채·무침류","젓갈류","김치류","장아찌·절임류"]):
        return "반찬류"
    elif any(x in row['식품대분류명'] for x in ["빵 및 과자류", "음료 및 차류", "유제품류 및 빙과류","샌드위치","곡류, 서류 제품"]):
        return "디저트류"
    elif any(x in row['식품대분류명'] for x in ["브런치","샌드위치"]):  # 브런치
        return "브런치류"
    else:
        return "기타"

filtered_food_data['음식분류'] = filtered_food_data.apply(classify_food, axis=1)

# 결과 확인
print("음식 분류 결과:")
print(filtered_food_data['음식분류'].value_counts())



음식 분류 결과:
음식분류
디저트류    935
반찬류     883
브런치류    558
밥류      328
국류      325
기타       41
Name: count, dtype: int64


<ipython-input-455-22e689c51923>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_food_data['음식분류'] = filtered_food_data.apply(classify_food, axis=1)


#머신러닝 모델 학습

#3. 데이터 준비

In [456]:
# 학습 데이터 생성
X = filtered_food_data[['칼로리_100g', '탄수화물_100g', '단백질_100g', '지방_100g']]
y = filtered_food_data['식품명']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


#3-1. 모델 학습


In [457]:
# KNN 모델 생성 및 학습
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
print("모델 학습 완료!")


모델 학습 완료!


#추천 시스템 구축


# 4.사용자 입력 및 칼로리 계산

In [458]:
# BMI 상태 계산 함수
def calculate_bmi(weight, height):
    bmi = weight / ((height / 100) ** 2)  # 키를 cm에서 m로 변환하여 계산
    if bmi < 18.5:
        return '저체중', bmi
    elif 18.5 <= bmi < 23:
        return '정상체중', bmi
    elif 23 <= bmi < 25:
        return '과체중', bmi
    else:
        return '비만', bmi

# BMR 계산 함수 (남성과 여성 구분)
def calculate_bmr(weight, height, age, gender):
    if gender == 'Male':  # 남성
        bmr = 10 * weight + 6.25 * height - 5 * age + 5
    elif gender == 'Female':  # 여성
        bmr = 10 * weight + 6.25 * height - 5 * age - 161
    else:
        raise ValueError("성별은 'Male' 또는 'Female'로 입력해야 합니다.")
    return bmr

# 사용자 입력 예시
user_weight = 50  # kg
user_height = 165  # cm
user_age = 24  # 세.    기초대사량 계산을 위해 나이도 있어야함
user_gender = 'Female'  # 성별
user_goal = '다이어트'  # 목표: 다이어트, 유지, 벌크업

# BMI 상태 자동 계산
user_bmi_class, user_bmi = calculate_bmi(user_weight, user_height)

# 사용자 BMR 및 칼로리 목표 계산
user_bmr = calculate_bmr(user_weight, user_height, user_age, user_gender)
user_calorie_target = get_calorie_target(user_bmi_class, user_bmr, user_goal)

# 목표별 영양소 비율 설정
if user_goal == '다이어트':
    # 다이어트 기준: 탄수화물 50%, 단백질 30%, 지방 20%
    carb_ratio, protein_ratio, fat_ratio = 0.5, 0.3, 0.2
elif user_goal == '유지':
    # 유지 기준: 탄수화물 50%, 단백질 25%, 지방 25%
    carb_ratio, protein_ratio, fat_ratio = 0.5, 0.25, 0.25
elif user_goal == '벌크업':
    # 벌크업 기준: 탄수화물 55%, 단백질 30%, 지방 15%
    carb_ratio, protein_ratio, fat_ratio = 0.55, 0.3, 0.15

# 영양소 목표 계산
user_carb_target = (user_calorie_target * carb_ratio) / 4   # 탄수화물: 1g = 4kcal
user_protein_target = (user_calorie_target * protein_ratio) / 4  # 단백질: 1g = 4kcal
user_fat_target = (user_calorie_target * fat_ratio) / 9    # 지방: 1g = 9kcal

# 출력
print(f"사용자 BMI: {user_bmi:.2f}")
print(f"BMI 상태: {user_bmi_class}")
print(f"목표: {user_goal}")
print(f"칼로리 목표: {user_calorie_target:.2f} kcal")
print(f"탄수화물 목표: {user_carb_target:.2f} g")
print(f"단백질 목표: {user_protein_target:.2f} g")
print(f"지방 목표: {user_fat_target:.2f} g")


사용자 BMI: 18.37
BMI 상태: 저체중
목표: 다이어트
칼로리 목표: 950.25 kcal
탄수화물 목표: 118.78 g
단백질 목표: 71.27 g
지방 목표: 21.12 g


#4-1. 메뉴 추천 함수

In [459]:
import random

# 끼니별 비율 설정
meal_ratios = {
    "breakfast": 0.3,  # 아침 30%
    "lunch": 0.4,      # 점심 40%
    "dinner": 0.3      # 저녁 30%
}

# 간식 추천 비율 설정 (목표에 따라 간식 추천 비중 변화)
snack_ratios = {
    "다이어트": {"morning_snack": 0.05, "afternoon_snack": 0.05, "evening_snack": 0.0},
    "유지": {"morning_snack": 0.1, "afternoon_snack": 0.1, "evening_snack": 0.05},
    "벌크업": {"morning_snack": 0.1, "afternoon_snack": 0.1, "evening_snack": 0.1},
}

# 사용자 목표별 영양소 비율
goal_ratios = {
    "다이어트": {"carb_ratio": 0.5, "protein_ratio": 0.3, "fat_ratio": 0.2},
    "유지": {"carb_ratio": 0.55, "protein_ratio": 0.25, "fat_ratio": 0.2},
    "벌크업": {"carb_ratio": 0.6, "protein_ratio": 0.3, "fat_ratio": 0.1},
}

# 간식 추천 여부 결정
def should_recommend_snack(goal, snack_type, calorie_target, used_calories):
    """
    간식 추천 여부를 결정.
    """
    snack_ratio = snack_ratios[goal][snack_type]
    if snack_ratio == 0.0:  # 간식을 추천하지 않음
        return False
    snack_calorie_limit = calorie_target * snack_ratio
    return used_calories + snack_calorie_limit <= calorie_target

# 식사 추천 함수: (밥류 + 국류 + 반찬류) 또는 브런치류
def recommend_meal_with_alternate_structure(calorie_target, food_data, used_foods, carb_ratio, protein_ratio, fat_ratio, last_meal_type=None):
    """
    식사 추천: (밥류 + 국류 + 반찬류)와 브런치류를 번갈아 추천.
    """
    # 영양소 목표 계산
    carb_target = calorie_target * carb_ratio / 4  # 탄수화물 g
    protein_target = calorie_target * protein_ratio / 4  # 단백질 g
    fat_target = calorie_target * fat_ratio / 9  # 지방 g

    # 초기화
    recommendations = []
    total_calories = 0
    total_carb = 0
    total_protein = 0
    total_fat = 0
    total_weight = 0  # 전체 섭취량

    # 항목별 최대 섭취량 설정
    portion_limits = {
        "밥류": 300,  # 최대 300g
        "국류": 550,  # 최대 250g
        "반찬류": 200,  # 최대 200g
        "브런치류": 4500,  # 최대 400g
    }

    # 추천 타입 결정: 번갈아 추천
    if last_meal_type == "브런치류" or last_meal_type is None:
        meal_type = "밥류+국류+반찬류"
    else:
        meal_type = "브런치류"

    if meal_type == "브런치류":
        # 브런치류 단일 추천
        brunch_data = food_data[(food_data['음식분류'] == '브런치류') & (~food_data['식품명'].isin(used_foods))]
        if not brunch_data.empty:
            brunch = brunch_data.sample(1).iloc[0]
            brunch_portion = min(
                calorie_target / brunch['칼로리_100g'] * 100,
                portion_limits["브런치류"]
            )
            used_foods.append(brunch['식품명'])
            return [f"{brunch['식품명']} {brunch_portion:.1f}g"], "브런치류"
        else:
            meal_type = "밥류+국류+반찬류"  # 브런치류가 없으면 다른 타입으로 대체

    if meal_type == "밥류+국류+반찬류":
        # (밥류 + 국류 + 반찬류) 추천
        for category in ["밥류", "국류", "반찬류"]:
            filtered_data = food_data[
                (food_data['음식분류'] == category) & (~food_data['식품명'].isin(used_foods))
            ]

            if filtered_data.empty:
                recommendations.append(f"추천 가능한 {category} 없음")
                continue

            food = filtered_data.sample(1).iloc[0]
            max_portion = portion_limits[category]
            portion = min(
                calorie_target / food['칼로리_100g'] * 100,
                max_portion
            )

            total_calories += food['칼로리_100g'] * (portion / 100)
            total_carb += food['탄수화물_100g'] * (portion / 100)
            total_protein += food['단백질_100g'] * (portion / 100)
            total_fat += food['지방_100g'] * (portion / 100)
            total_weight += portion

            recommendations.append(f"{food['식품명']} {portion:.1f}g")
            used_foods.append(food['식품명'])

            # 목표 충족 여부 확인
            if (
                total_calories >= calorie_target * 0.9 and total_calories <= calorie_target * 1.1 and
                total_carb >= carb_target * 0.9 and
                total_protein >= protein_target * 0.9 and
                total_fat >= fat_target * 0.9
            ):
                break  # 목표 충족 시 종료

    return recommendations, "밥류+국류+반찬류"


# 간식 추천 함수 (3가지 선택지 제공, 섭취량이 비정상적으로 큰 경우 "마음껏" 출력)
def recommend_snack_choices(calorie_target, food_data, used_foods, num_choices=3, max_portion=1000):
    """
    간식 추천: 디저트류에서 num_choices 개를 랜덤으로 추천하며, 섭취량 계산.
    섭취량이 max_portion을 초과하면 '마음껏'으로 출력.
    """
    snack_data = food_data[food_data['음식분류'] == '디저트류']
    valid_snacks = snack_data[~snack_data['식품명'].isin(used_foods)]

    if valid_snacks.empty:
        return "추천 가능한 간식 없음"

    # 랜덤으로 num_choices 개 선택
    snack_choices = valid_snacks.sample(min(num_choices, len(valid_snacks)))
    recommendations = []
    for _, snack in snack_choices.iterrows():
        portion = calorie_target / snack['칼로리_100g'] * 100  # 섭취량(g) 계산
        used_foods.append(snack['식품명'])

        # 섭취량이 max_portion 초과 시 '마음껏' 출력
        if portion > max_portion:
            recommendations.append(f"{snack['식품명']} 마음껏")
        else:
            recommendations.append(f"{snack['식품명']} {portion:.1f}g")

    return " or ".join(recommendations)


# 추천된 음식 목록 및 사용된 칼로리 기록
used_foods = []
used_calories = 0

# 목표별 영양소 비율 설정
carb_ratio = goal_ratios[user_goal]["carb_ratio"]
protein_ratio = goal_ratios[user_goal]["protein_ratio"]
fat_ratio = goal_ratios[user_goal]["fat_ratio"]

# 이전 끼니 타입 저장
last_meal_type = None

# 아침 식사 추천
breakfast, last_meal_type = recommend_meal_with_alternate_structure(user_calorie_target * meal_ratios["breakfast"],
                                                                    filtered_food_data, used_foods,
                                                                    carb_ratio, protein_ratio, fat_ratio,
                                                                    last_meal_type)
used_calories += user_calorie_target * meal_ratios["breakfast"]

# 아침-점심 간 간식 추천
morning_snack = None
if should_recommend_snack(user_goal, "morning_snack", user_calorie_target, used_calories):
    morning_snack = recommend_snack_choices(user_calorie_target * snack_ratios[user_goal]["morning_snack"],
                                            filtered_food_data, used_foods)
    used_calories += user_calorie_target * snack_ratios[user_goal]["morning_snack"]

# 점심 식사 추천
lunch, last_meal_type = recommend_meal_with_alternate_structure(user_calorie_target * meal_ratios["lunch"],
                                                                filtered_food_data, used_foods,
                                                                carb_ratio, protein_ratio, fat_ratio,
                                                                last_meal_type)
used_calories += user_calorie_target * meal_ratios["lunch"]


# 점심-저녁 간 간식 추천
afternoon_snack = None
if should_recommend_snack(user_goal, "afternoon_snack", user_calorie_target, used_calories):
    afternoon_snack = recommend_snack_choices(user_calorie_target * snack_ratios[user_goal]["afternoon_snack"],
                                              filtered_food_data, used_foods)
    used_calories += user_calorie_target * snack_ratios[user_goal]["afternoon_snack"]

# 저녁 식사 추천
dinner, last_meal_type = recommend_meal_with_alternate_structure(user_calorie_target * meal_ratios["dinner"],
                                                                 filtered_food_data, used_foods,
                                                                 carb_ratio, protein_ratio, fat_ratio,
                                                                 last_meal_type)
used_calories += user_calorie_target * meal_ratios["dinner"]

# 저녁 이후 간식 추천
evening_snack = None
if should_recommend_snack(user_goal, "evening_snack", user_calorie_target, used_calories):
    evening_snack = recommend_snack_choices(user_calorie_target * snack_ratios[user_goal]["evening_snack"],
                                            filtered_food_data, used_foods)
    used_calories += user_calorie_target * snack_ratios[user_goal]["evening_snack"]

# 결과 출력
print("추천 식단:")
print(f"아침: {breakfast}\n")
if morning_snack:
    print(f"간식 (아침-점심 사이): {morning_snack}\n")
print(f"점심: {lunch}\n")
if afternoon_snack:
    print(f"간식 (점심-저녁 사이): {afternoon_snack}\n")
print(f"저녁: {dinner}\n")
if evening_snack:
    print(f"간식 (저녁 이후): {evening_snack}\n")


추천 식단:
아침: ['짬뽕_간편조리세트_통오징어짬뽕탕 300.0g', '김치찌개_참치 550.0g', '감자볶음_감자 200.0g']

간식 (아침-점심 사이): 커피_디카페인 헤이즐넛아메리카노 핫(HOT) 879.9g or 샌드위치_채소 35.1g or 커피_디카페인 허니아메리카노 아이스(ICED) 마음껏

점심: ['피자_뉴욕 오리진 피자 오리지널 (M) 1008.0g']

간식 (점심-저녁 사이): 도넛_찹쌀 11.2g or 페이스트리_프렌치뀐아망 6.7g or 파이/만주_달콤한딸기롤파이 (3개입) 12.1g

저녁: ['회덮밥_모듬 300.0g', '마라탕_간편조리세트_마라탕 550.0g', '돼지갈비구이_간장 200.0g']

